In [1]:
import pandas as pd 
import json 
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf 
from keras.callbacks import EarlyStopping
import tensorflow_hub as hub 
import tensorflow_text as text
from tensorflow.keras import optimizers

In [2]:
df = pd.read_csv('IBC.csv', index_col=False)
df

,SENTENCE,LABEL
0,Forcing middle-class workers to bear a greater...,1
1,Because it would not be worthwhile to bring a ...,1
2,"Indeed , Lind argues that high profits and hig...",1
3,"In fairness , it should be noted that he devot...",1
4,Psychological tactics are social control techn...,1
...,...,...
4321,"As Doug Ogden , former director of the Energy ...",0
4322,No study is perfect ; each one is subject to c...,0
4323,"Of course , market forces , the balancing of ,...",0
4324,"In Words Like Loaded Pistols , he sets out to ...",0


In [3]:
df.sentences = df.SENTENCE.str.lower()
df

C:\Users\Mrulay\AppData\Local\Temp\ipykernel_25640\41936758.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.sentences = df.SENTENCE.str.lower()


,SENTENCE,LABEL
0,Forcing middle-class workers to bear a greater...,1
1,Because it would not be worthwhile to bring a ...,1
2,"Indeed , Lind argues that high profits and hig...",1
3,"In fairness , it should be noted that he devot...",1
4,Psychological tactics are social control techn...,1
...,...,...
4321,"As Doug Ogden , former director of the Energy ...",0
4322,No study is perfect ; each one is subject to c...,0
4323,"Of course , market forces , the balancing of ,...",0
4324,"In Words Like Loaded Pistols , he sets out to ...",0


In [4]:
df['SENTENCE'] = df['SENTENCE'].str.replace(r'[^\w\s]+', '')
df

C:\Users\Mrulay\AppData\Local\Temp\ipykernel_25640\3792878244.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['SENTENCE'] = df['SENTENCE'].str.replace(r'[^\w\s]+', '')


,SENTENCE,LABEL
0,Forcing middleclass workers to bear a greater ...,1
1,Because it would not be worthwhile to bring a ...,1
2,Indeed Lind argues that high profits and high...,1
3,In fairness it should be noted that he devote...,1
4,Psychological tactics are social control techn...,1
...,...,...
4321,As Doug Ogden former director of the Energy F...,0
4322,No study is perfect each one is subject to cr...,0
4323,Of course market forces the balancing of sa...,0
4324,In Words Like Loaded Pistols he sets out to s...,0


In [5]:
stop = stopwords.words('english')
df['SENTENCE'] = df['SENTENCE'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
df

,SENTENCE,LABEL
0,forcing middleclass workers bear greater share...,1
1,because would worthwhile bring case 3022 arbit...,1
2,indeed lind argues high profits high wages rei...,1
3,in fairness noted devotes entire chapter new y...,1
4,psychological tactics social control technique...,1
...,...,...
4321,as doug ogden former director energy foundatio...,0
4322,no study perfect one subject criticism illegit...,0
4323,of course market forces balancing say demand s...,0
4324,in words like loaded pistols sets share enthus...,0


In [6]:
labels = pd.get_dummies(df['LABEL'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['SENTENCE'],labels)

In [16]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", trainable=True)
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)

In [17]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(3, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ERROR:absl:hub.KerasLayer is trainable but has zero trainable weights.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [19]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

learning_rate = 0.0001
optimizer = optimizers.Adam(learning_rate)
    
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=METRICS)

In [20]:
es = EarlyStopping(
    monitor='accuracy',
    patience=2,
    min_delta = 0.1,
    mode='max'
)

In [21]:
model.fit(X_train, y_train, epochs=20, verbose=1, shuffle=True, callbacks=[es])

Epoch 1/20
 32/102 [========>.....................] - ETA: 15:06 - loss: 0.6049 - accuracy: 0.6445 - precision: 0.4408 - recall: 0.2471

KeyboardInterrupt: 

In [14]:
model.evaluate(X_test, y_test)

34/34 [==============================] - 142s 4s/step - loss: 0.5661 - accuracy: 0.6873 - precision: 0.5548 - recall: 0.3133


[0.566116452217102, 0.6873074173927307, 0.5548281669616699, 0.3133086860179901]

In [15]:
model.save('IBC_BERT')

INFO:tensorflow:Assets written to: IBC_BERT\assets


INFO:tensorflow:Assets written to: IBC_BERT\assets
